In [1]:
from bs4 import BeautifulSoup
import re, time

import urllib.request

class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

def getTextFromArticle(link):
    try:
        opener = AppURLopener()
        response = opener.open(link).read()
        soup = BeautifulSoup(response, "lxml")
        dataHtml = soup.find("div", {"itemprop": "articleBody"})

        #print(dataHtml.getText())
        text=dataHtml.getText()
        #print(type(text))

        #Remove links
        regex=r'https([^ ]+)'
        t=re.sub(regex, '', text)
        
        #Same for http starting
        regex=r'http([^ ]+)'
        t=re.sub(regex, '', text)
        
        #Same for files
        #regex=r'file:([^ ]+)'
        #t=re.sub(regex, '', text)
        
        
        #print(t)
        #Remove ads
        ad_list=[r'Mainos\s\(Teksti jatkuu alla\)',
                 r'try {',
                 r"sanomaAd.+",
                 r'catch\s\(err\).*',
                 r'try+',
                 r'{',
                 r'}',
                 r'\s\s\s\s\s\s\s\s\s\s\s\s\s\s']
        for word in ad_list:
            #print(word)
            t=re.sub(word, '', t)

        text=t.replace("Mainos päättyy","")

        processed_text=re.sub(r'\s\s', ' ', text)
        #print(processed_text)
        return processed_text
    except:
        print("Could not get text from article: ",link)
        return "Error"

link="https://www.hs.fi/urheilu/art-2000005974075.html"
getTextFromArticle(link)
print(getTextFromArticle(link))

/usr/lib/python3/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  # This is added back by InteractiveShellApp.init_path()



Kreikkalainen tennissensaatio Stefanos Tsitsipas on tottunut satuilemaan, ja nyt hän tekee sitä myös tenniskentällä Australian mestaruusturnauksessa.Tsitsipas, 20, antoi lisää kierroksia lajin nuorisovallankumoukselle voittamalla neljännellä kierroksella Sveitsin Roger Federerin Lisää vauhtia tuli, kun hän kukisti puolivälierissä Espanjan Roberto Bautista Agutin 7–5, 4–6, 6–4, 7–6 (2). ”Tämä tuntuu ihan sadulta”, Tsitsipas sanoi Jim Courierin kenttähaastattelussa.Federer on ollut Tsitsipasin idoli varhaisesta lapsuudesta lähtien, ja nyt hän saattoi siirtää lajilegendan syrjään grand slamien voittotaisteluista pysyvästi. Pelkästään Federerin voittaminen pöllyttää tenniksen valtarakenteita uusiksi, ja samaa Tsitsipas tekee videobloggaajana.Tsitsipas kuvaa tiheään tahtiin videoblogeja ja paljastaa elämänsä ja lähipiirinsä ihmisten katseltavaksi.Tubettava tennisammattilainen kertoo meille, että New York on suosikkikaupunki, NBA-koripallon seuraaminen yksi intohimoista ja perhe ympärillä e

In [2]:
def getDataFromLink(link):
    opener = AppURLopener()
    response = opener.open(link).read()
    soup = BeautifulSoup(response, "lxml")
    dataHtml = soup.findAll("div")
    
    #links = [a.find("a") for a in dataHtml]
    #print(links)
    #teasers=soup.find_all("h2")
    teasers=soup.findAll("article", {"class": "teaser full-width fast"})
    data=[]
    for t in teasers:
        link="https://hs.fi"+str(t.find("a")['href'])
        teaser=str(t.find("h2")).replace("<h2>","").replace("</h2>","")
        teaser=re.sub("<span.*?span>", "", teaser)
        article_text=getTextFromArticle(link)
        #print(link)
        #print(teaser)
        #getTextFromLink(link)
        data.append([link,teaser,article_text])
        #break
    #data = dataHtml.find("li", {"class": "bold"})
    #print(teasers[0])
    #print(soup)
    #print(rawList)
    return data

import csv
def writeToCsv(file,data):
    with open(file,'a',encoding='utf8') as f:
        wr = csv.writer(f, dialect='excel',delimiter=';')
        wr.writerows(data)

link='https://hs.fi/'
data=getDataFromLink(link)
for d in data:
    print(d)
#writeToCsv("hsData.csv", data)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  # This is added back by InteractiveShellApp.init_path()


['https://hs.fi/kaupunki/espoo/art-2000006020151.html', '\n\nLeppävaaran aseman opastekyltit asennettiin väärien laitureiden kohdalle ja ylösalaisin\n', '\nPerjantaina Leppävaaran asemalla junaan suunnistaneet saivat hieraista silmiään. Laiturille kaksi vievien portaiden alkupäässä oli valtava opastekyltti, jossa oli numero neljä – ylösalaisin. Sama näky toistui myös laiturille neljä vievien portaiden alkupäässä: siellä oli ylösalaisin kyltti, jossa luki numero kaksi.”Oho! Täh...tässä on varmasti tapahtunut työvirhe.” Tämä oli alikulkutunnelin toteutuksesta Espoon kaupungilla vastaavan projektin vetäjän, maisema-arkkitehti Kristina Rochan ensimmäinen reaktio, kun hän kuuli kummallisista kylteistä.”Kieltämättä tämä herättää kysymyksiä. Ei niitä varmasti noin ole ollut tarkoitus asentaa”, Rocha sanoo. Kylttien asennuksesta vastaa VRJ Etelä-Suomi. Myös siellä oltiin kylttien asennuksesta ihmeissään.”Siinä on ollut kommunikointihaaste kahden urakoitsijan välillä. Tietokatkon vuoksi teippau

In [4]:
import datetime

dir_name="./hs_data/"
time=str(datetime.datetime.now().date())
filename=time + " hs data.csv"
f=open(dir_name+filename,'a',encoding='utf8')
f.close()

raw_link='https://www.hs.fi'
print("Writing to file "+dir_name+filename)
failed=0

link=raw_link
data=getDataFromLink(link)
writeToCsv(dir_name+filename, data)
    
print("Finished")

Writing to file ./hs_data/2019-03-02 hs data.csv


/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  # This is added back by InteractiveShellApp.init_path()


Finished
